In [1]:
import os
import time
import json
import traceback

import requests
from tqdm import tqdm

In [2]:
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
    'sec-ch-ua-platform': '"macOS"',
    'Accept': '*/*',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'no-cors',
    'Sec-Fetch-Dest': 'script',
    'Referer': 'https://maplocation.sjfkai.com/',
    'Accept-Language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
}

In [19]:
def get_address(addr, timeout=30):
    params = {
        'address': addr,
        'output': 'json',
        'ak': 'your_ak',
        'callback': 'showLocation0',
    }

    try:
        response = requests.get(
            'https://api.map.baidu.com/geocoding/v3/', headers=headers, params=params, timeout=timeout)

        obj = json.loads(response.text[len('showLocation0&&showLocation0('):-1])

        if obj['status'] == 0:
            return obj['result']['location']
    except:
        traceback.print_exc()

In [20]:
get_address('北京市')

{'lng': 116.4133836971231, 'lat': 39.910924547299565}

In [4]:
addr = json.load(open('addr.json'))

In [5]:
addr_flatten = []
for x in addr:
    addr_flatten += x['addr']
addr_flatten = list(set(addr_flatten))
print(len(addr_flatten))

17986


In [6]:
addr_flatten[:3]

['浦东新区汇南村', '青浦区新桥路761弄', '黄浦区自忠路373号,']

In [7]:
addr_cache_path = 'addr_cache.json'

if os.path.exists(addr_cache_path):
    addr_cache = json.load(open(addr_cache_path))
else:
    addr_cache = {}

for a in tqdm(addr_flatten):
    if a in addr_cache:
        continue
    lng_lat = get_address(a)
    if lng_lat is not None:
        addr_cache[a] = lng_lat
    time.sleep(0.1)

with open(addr_cache_path, 'w') as fp:
    json.dump(addr_cache, fp, indent=4, ensure_ascii=False)

100%|██████████| 17986/17986 [51:14<00:00,  5.85it/s]  


In [8]:
print(len(addr_cache))

17985


In [9]:
!head addr_cache.json

{
    "杨浦区包头南路588弄": {
        "lng": 121.54687836276618,
        "lat": 31.30610017684735
    },
    "嘉定区年丰村": {
        "lng": 121.27896611052044,
        "lat": 31.26423830382887
    },
    "嘉定区汇仁路1988号": {


In [10]:
!tail addr_cache.json

    },
    "浦东新区民乐村": {
        "lng": 121.74050056967565,
        "lat": 31.07019233238509
    },
    "嘉定区嘉怡路329号": {
        "lng": 121.35431373093361,
        "lat": 31.270996506291954
    }
}